<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN05/5.2-Von_MLPs_zu_CNNs.ipynb)

# Von MLPs zu CNNs


Die Modelle, die wir bisher diskutiert haben sind valide Optionen, wenn wir es mit **tabellarischen Daten** zu tun haben.
- Mit tabellarisch meinen wir, dass die Daten aus aus Zeilen, die den Beispielen entsprechen und Spalten, die den Merkmalen entsprechen (tidy data).
- Bei tabellarischen Daten gehen wir davon aus, dass die gesuchten Muster **Wechselwirkungen zwischen den Merkmalen** darstellen, aber wir gehen nicht von einer Struktur *a priori* aus darüber an, wie die Merkmale interagieren.
- Momentan fehlt uns das Wissen für die Konstruktion ausgefeilterer Architekturen. In diesen Fällen kann  ein **MLP** das Beste sein, was wir tun können. Doch bei hochdimensionalen Wahrnehmungsdaten können solche strukturlosen Netze unhandlich oder undurchführbar werden.

**Ein Beispiel:** Nehmen Sie an, wir sammeln einen annotierten Datensatz von Ein-Megapixel-Fotos von Katzen und Hunden und möchten mit diesen Daten einen MLP-Klassifizierer bauen. 
- Das bedeutet, dass jede Eingabe in das Netz eine Million Dimensionen hat. Selbst eine aggressive Reduzierung auf eintausend versteckte Dimensionen würde eine **voll vernetzte Schicht erfordern mit $10^6 \times 10^3 = 10^9$ Parametern !** 
- Es sei denn, wir haben viele GPUs, ein Talent für verteilte Optimierung, und eine außergewöhnliche Menge an Geduld, kann sich das Lernen der Parameter dieses Netzes sich als undurchführbar erweisen.

Und doch sind heute sowohl Menschen als auch Computer in der Lage in der Lage, Katzen und Hunde recht gut zu unterscheiden,
was scheinbar im Widerspruch zu diesen Intuitionen steht. Das liegt daran, dass Bilder eine reichhaltige Struktur aufweisen
die sowohl Menschen als auch von und maschinelle Lernmodelle gleichermaßen nutzen können.
Faltungsneuronale Netze (CNNs) sind eine kreativer Weg, den das maschinelle Lernen eingeschlagen hat, um einige der bekannten **Strukturen in natürlichen Bildern** zu nutzen.


## Invarianz

- Stellen Sie sich vor, Sie möchten ein Objekt in einem Bild erkennen.
- Es scheint vernünftig, dass jede Methode, die wir zur Erkennung von Objekten verwenden, sich nicht zu sehr um die genauen Position des Objekts im Bild kümmert (**Translationsinvarianz**).
- Im Idealfall sollte unser System dieses Wissen ausnutzen. Schweine fliegen normalerweise nicht und Flugzeuge schwimmen normalerweise nicht. Nichtsdestotrotz sollten wir ein Schwein erkennen, wenn es am oberen Rand des Bildes erscheint.

Hier können wir uns ein wenig vom Kinderspiel "Wo ist Waldo?" inspirieren lassen. 

Das Spiel besteht aus einer Reihe von chaotischen Szenen, die vor Aktivitäten strotzen. Waldo taucht irgendwo in jeder Szene auf, typischerweise lauert er an einem unwahrscheinlichen Ort. Das Ziel des Lesers ist es, ihn zu finden.
Trotz seines charakteristischen Outfits, kann dies erstaunlich schwierig sein, aufgrund der großen Anzahl von Ablenkungen.
Doch *wie Waldo aussieht* hängt nicht davon ab, *wo sich Waldo befindet*.


Wir könnten das Bild mit einem Waldo-Detektor durchforsten der jedem Fleck eine Punktzahl zuweisen könnte,
der die Wahrscheinlichkeit angibt, dass der Fleck Waldo enthält.
CNNs systematisieren diese Idee der *Translationsinvarianz*, und nutzen sie, um nützliche Darstellungen mit weniger Parametern zu lernen.

<img src="Bilder/where-wally-walker-books.jpg" width="640" height="440" align="center"/>




### Anforderungen: Translationsinvarianz und Lokalität

Wir können diese Intuitionen nun konkreter machen, indem wir einige Anforderungen aufzählen, die unser Design leiten sollen
eines neuronalen Netzes, das für die Computer Vision geeignet ist:

1. In den ersten Schichten sollte unser Netz ähnlich auf denselben Fleck reagieren, unabhängig davon, wo er im Bild erscheint. Dieses Prinzip wird **Translationsinvarianz** genannt.
2. Die frühesten Schichten des Netzes sollten sich auf lokale Regionen konzentrieren, ohne Rücksicht auf den Inhalt des Bildes in entfernten Regionen. Dies ist das Prinzip der **Lokalität**. Letztendlich können diese lokalen Repräsentationen aggregiert werden, um Vorhersagen auf der gesamten Bildebene zu treffen.


## Beschränkung des MLP

Für den Anfang können wir einen MLP betrachten mit zweidimensionalen Bildern $\mathbf{X}$ als Eingaben und ihren unmittelbaren versteckten Repräsentationen $\mathbf{H}$, die in der Mathematik als Matrizen und im Code als zweidimensionale Tensoren dargestellt werden, wobei sowohl $\mathbf{X}$ als auch $\mathbf{H}$ die gleiche Form haben.

Wir stellen uns nun vor, dass nicht nur die Eingaben, sondern sondern auch die verborgenen Repräsentationen als räumlich strukturiert.

Bezeichnen $[\mathbf{X}]_{i, j}$ und $[\mathbf{H}]_{i, j}$ die Pixel an der Stelle ($i$, $j$)
im Eingabebild bzw. in der versteckten Darstellung. Damit jede der versteckten Einheiten
Input von jedem der Eingangspixel erhält, würden wir von der Verwendung von Gewichtsmatrizen zur Darstellung unserer Parameter
als Gewichtstensoren vierter Ordnung $\mathsf{W}$ darstellen. 

Nehmen wir an, dass $\mathbf{U}$ Offsets (bias terms) enthält,
könnten wir die voll verknüpfte Schicht formal ausdrücken als

$$\begin{aligned} \left[\mathbf{H}\right]_{i, j} &= [\mathbf{U}]_{i, j} + \sum_k \sum_l[\mathsf{W}]_{i, j, k, l}  [\mathbf{X}]_{k, l}\\ &= [\mathbf{U}]_{i, j} +
\sum_a \sum_b [\mathsf{V}]_{i, j, a, b} [\mathbf{X}]_{i+a, j+b}.\end{aligned},$$

wobei der Wechsel von $\mathsf{W}$ zu $\mathsf{V}$ vorerst rein kosmetisch ist, da es eine Eins-zu-Eins-Entsprechung
zwischen den Koeffizienten der beiden Tensoren vierter Ordnung besteht.

Wir indexieren einfach die Indizes $(k, l)$ neu so dass $k = i+a$ und $l = j+b$.
Mit anderen Worten: Wir setzen $[\mathsf{V}]_{i, j, a, b} = [\mathsf{W}]_{i, j, i+a, j+b}$.
Die Indizes $a$ und $b$ laufen sowohl über positive als auch negative Offsets,
und decken das gesamte Bild ab.
Für jede beliebige Stelle ($i$, $j$) in der verborgenen Darstellung $[\mathbf{H}]_{i, j}$,
berechnen wir seinen Wert durch Summation über die Pixel in $x$,
die um $(i, j)$ zentriert und mit $[\mathsf{V}]_{i, j, a, b}$ gewichtet sind.

### Translationsinvarianz

Nun wollen wir uns auf das erste Prinzip berufen, das oben aufgestellt wurde: die Translationsinvarianz.
Dies impliziert, dass eine Verschiebung der Eingabe $\mathbf{X}$ einfach zu einer Verschiebung in der verborgenen Darstellung $\mathbf{H}$ führen sollte.
Dies ist nur möglich, wenn $\mathsf{V}$ und $\mathbf{U}$ tatsächlich nicht von $(i, j)$ abhängen,
d.h. wir haben $[\mathsf{V}]_{i, j, a, b} = [\mathbf{V}]_{a, b}$ und $\mathbf{U}$ ist eine Konstante, sagen wir $u$.
Daher können wir die Definition für $\mathbf{H}$ vereinfachen:

$$[\mathbf{H}]_{i, j} = u + \sum_a\sum_b [\mathbf{V}]_{a, b} [\mathbf{X}]_{i+a, j+b}.$$


Dies ist eine *Faltung*!
Wir gewichten effektiv Pixel an $(i+a, j+b)$
in der Nähe der Position $(i, j)$ mit den Koeffizienten $[\mathbf{V}]_{a, b}$
um den Wert $[\mathbf{H}]_{i, j}$ zu erhalten.
Man beachte, dass $[\mathbf{V}]_{a, b}$ viel weniger Koeffizienten benötigt als $[\mathsf{V}]_{i, j, a, b}$, da es
nicht mehr von der Position innerhalb des Bildes abhängt.
Wir haben bedeutende Fortschritte gemacht!

### Lokalität

Nun wollen wir uns auf den zweiten Grundsatz berufen: die Lokalität.
Wie oben begründet, glauben wir, dass wir nicht nicht sehr weit vom Ort $(i, j)$ wegschauen müssen
um relevante Informationen zu erhalten um zu beurteilen, was an $[\mathbf{H}]_{i, j}$ vor sich geht.
Das bedeutet, dass ausserhalb eines Bereichs $|a|> \Delta$ oder $|b| > \Delta$,
wir $[\mathbf{V}]_{a, b} = 0$ setzen sollten.

Äquivalent dazu können wir $[\mathbf{H}]_{i, j}$ umschreiben als

$$[\mathbf{H}]_{i, j} = u + \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} [\mathbf{V}]_{a, b} [\mathbf{X}]_{i+a, j+b}. \tag{1}$$

Beachten Sie, dass (1), kurz gesagt, eine *Faltungsschicht* ist.


### Faltungskerne 

*Konvolutionelle neuronale Netze* (CNNs) sind eine spezielle Familie von neuronalen Netzen, die Faltungsschichten enthalten.

In der Deep-Learning-Forschungsgemeinschaft,  wird $\mathbf{V}$ als *Faltungskern* oder *Filter* bezeichnet. 

Wenn die lokale Region klein ist, kann der Unterschied im Vergleich zu einem vollständig verbundenen Netz dramatisch sein.
Während wir früher möglicherweise Milliarden von Parametern benötigt haben, um nur eine einzige Schicht in einem Bildverarbeitungsnetz darzustellen, brauchen wir jetzt in der Regel nur noch ein paar hundert, ohne dass sich die Dimensionalität der Eingänge der Eingaben oder der versteckten Repräsentationen.


Der Preis für diese drastische Reduzierung der Parameter ist, dass unsere Merkmale jetzt nicht mehr übersetzungsinvariant sind
und dass unsere Schicht nur noch lokale Informationen aufnehmen kann,
wenn sie den Wert jeder versteckten Aktivierung bestimmt.




## Inductive Bias - Hypothesen des Datenmodells

- Jegliches Lernen hängt davon ab, dass man eine induktive Hypothese auferlegt, wie die Daten als Modell generiert werden könneten. 
- Wenn diese Hypothese mit der Realität übereinstimmt, erhalten wir stichprobeneffiziente Modelle, die sich gut auf ungesehene Daten verallgemeinern lassen.
- Wenn dieser induktive Bias jedoch nicht mit der Realität übereinstimmt, könnten unsere Modelle sogar Schwierigkeiten haben, sich an unsere Trainingsdaten anzupassen (z. B. wenn sich herausstellt, dass Bilder nicht translationsinvariant sind). 


## "Wo ist Waldo?" revisited

Kehren wir zu unserem Waldo-Detektor zurück und sehen wir uns an, wie dieser aussieht.
- Die Faltungsschicht wählt Fenster mit einer bestimmten Grösse und gewichtet die Intensitäten gemäss dem Filter $\mathsf{V}$, wie in der folgenden Abbilding gezeigt.
- Unser Ziel könnte es sein, ein Modell zu lernen, bei dem wo immer die "Waldoness" am höchsten ist, wir einen Peak in den Darstellungen der verborgenen Schicht finden sollten.

<img src="Bilder/waldo-mask.jpg" width="640" height="440" align="center"/>








## Kanäle (channles)

Es gibt nur ein Problem mit diesem Ansatz. 

- Bis jetzt haben wir geflissentlich ignoriert, dass Bilder aus aus 3 Kanälen bestehen: Rot, Grün und Blau.
- In Wirklichkeit sind Bilder keine zweidimensionalen Objekte, sondern vielmehr Tensoren dritter Ordnung, die durch eine Höhe, eine Breite und einen Kanal gekennzeichnet sind, z. B. mit der Form $1024 \times 1024 \times 3$ Pixel.
- Während die ersten beiden dieser Achsen räumliche Beziehungen betreffen, kann die dritte als Zuweisung einer mehrdimensionale Darstellung zu jeder Pixelposition interpretiert werden.

Wir indizieren $\mathsf{X}$ also als $[\mathsf{X}]_{i, j, k}$.

Der Faltungsfilter muss sich entsprechend anpassen. Anstelle von $[\mathbf{V}]_{a,b}$ haben wir nun $[\mathsf{V}]_{a,b,c}$.


- Da unsere Eingabe aus einem **Tensor dritter Ordnung** besteht, erweist es sich als eine gute Idee, unsere verborgenen Repräsentationen in ähnlicher Weise als Tensoren dritter Ordnung $\mathsf{H}$ zu formulieren.
- Mit anderen Worten: Anstatt nur eine einzige verborgene Darstellung zu haben, die jeder räumlichen Position entspricht, wollen wir einen ganzen Vektor von verborgenen Repräsentationen entsprechend jeder räumlichen Position.

Man könnte sich die verborgenen Repräsentationen als eine Reihe von übereinander gestapelten zweidimensionalen Gittern (Matrizen) vorstellen. 
- Wie bei den Eingaben werden diese manchmal als *Kanäle* (**channels**) bezeichnet.
- Manchmal werden sie auch **Merkmalskarten** genannt, da jede Schicht einen räumlichen Satz der gelernten Merkmale an die nachfolgende Schicht weitergibt. Intuitiv könnte man sich vorstellen, dass in den unteren Schichten, die näher an den Eingängen liegen, einige Kanäle auf die Erkennung von Kanten spezialisiert sind, während andere Texturen erkennen könnten.

Um mehrere Kanäle sowohl in den Eingaben ($\mathsf{X}$) als auch in den versteckten Darstellungen ($\mathsf{H}$) zu unterstützen,
können wir eine vierte Koordinate zu $\mathsf{V}$ hinzufügen: $[\mathsf{V}]_{a, b, c, d}$.
Setzt man alles zusammen, so erhält man:

$$[\mathsf{H}]_{i,j,d} = \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} \sum_c [\mathsf{V}]_{a, b, c, d} [\mathsf{X}]_{i+a, j+b, c},$$
:eqlabel:`eq_conv-Schicht-Kanäle`

wobei $d$ die Ausgangskanäle in den verborgenen Darstellungen $\mathsf{H}$ indiziert. Die nachfolgende Faltungsschicht nimmt einen Tensor dritter Ordnung, $\mathsf{H}$, als Eingabe.
Allgemeiner formuliert,
:eqref:`eq_conv-layer-channels` ist
die Definition einer Faltungsschicht für mehrere Kanäle, wobei $\mathsf{V}$ ein Kernel oder Filter der Schicht ist.

Es gibt noch viele Operationen, die wir angehen müssen.
Zum Beispiel müssen wir herausfinden, wie wir alle versteckten Repräsentationen
zu einer einzigen Ausgabe kombinieren, z. B. ob es *irgendwo* im Bild einen Waldo gibt.
Wir müssen auch entscheiden, wie wir die Dinge effizient berechnen können,
wie man mehrere Schichten kombinieren kann,
geeignete Aktivierungsfunktionen,
und wie man vernünftige Designentscheidungen trifft
um Netzwerke zu entwickeln, die in der Praxis effektiv sind.
Diesen Fragen widmen wir uns im weiteren Verlauf des Kapitels.


## Zusammenfassung

* **Translationsinvarianz** in Bildern bedeutet, dass alle Bereiche eines Bildes auf die gleiche Weise behandelt werden.
* **Lokalität** bedeutet, dass nur eine kleine Nachbarschaft von Pixeln verwendet wird, um die entsprechenden versteckten Darstellungen zu berechnen.
* In der Bildverarbeitung benötigen Faltungsschichten in der Regel viel weniger Parameter als voll verknüpfte Schichten.
* CNNS sind eine spezielle Familie von neuronalen Netzen, die Faltungsschichten enthalten.
* Kanäle am Eingang und Ausgang ermöglichen es unserem Modell, mehrere Aspekte eines Bildes an jeder räumlichen Position zu erfassen.




## Übungen (optional)

1. Nehmen Sie an, dass die Grösse des Faltungskerns $\Delta = 0$ ist.
   Zeigen Sie, dass in diesem Fall der Faltungskern ein MLP unabhängig für jeden Satz von Kanälen implementiert.
2. Warum ist Translationsinvarianz vielleicht doch keine so gute Idee?
3. Mit welchen Problemen müssen wir uns auseinandersetzen, wenn wir entscheiden, wie versteckte Repräsentationen zu behandeln sind, die den Pixelpositionen an den Rändern eines Bildes entsprechen?
4. Beschreiben Sie eine Faltungsschicht für Audio-Daten.
5. Glauben Sie, dass Faltungsschichten auch für Textdaten geeignet sind? Warum oder warum nicht?